In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [11]:
load_dotenv()

True

In [12]:
#Loading my Bybit's API keys from the dotenv file
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [13]:
#Stablishing Connection with the API (SPOT)
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

#Creating a Dataframe of ETH real price (1m)
frame = pd.DataFrame(session_auth.query_kline(symbol="ETHUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1675804500000,1668.8,1668.8,1667.07,1667.08,12.24553,1675804560000,20416.9159334,54,0,0
1,1675804560000,1667.08,1667.08,1666.32,1666.88,29.1376,1675804620000,48564.8431321,61,0,0
2,1675804620000,1666.88,1666.88,1666.34,1666.34,9.11783,1675804680000,15197.454603,19,0,0
3,1675804680000,1666.34,1666.62,1666.32,1666.32,8.27792,1675804740000,13794.7771723,31,0,0
4,1675804740000,1666.32,1666.78,1666.32,1666.58,13.79786,1675804800000,22995.4663002,41,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1675864200000,1675.01,1675.33,1674.5,1674.5,2.21562,1675864260000,3711.5070738,28,0,0
996,1675864260000,1674.5,1675.21,1674.5,1675.21,1.32049,1675864320000,2211.4928777,20,0,0
997,1675864320000,1675.21,1675.21,1673.72,1673.91,17.21808,1675864380000,28820.9603847,45,0,0
998,1675864380000,1673.91,1674.95,1673.79,1674.95,27.04372,1675864440000,45275.2064882,66,0,0


In [14]:
#This function gets Real BTC Price Data and creates a smooth dataframe that refreshes every 15 minutes
def get5minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="ETHUSDT", interval="5m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [15]:
df = get5minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-02-05 02:35:00,1666.19,1667.39,1665.45,1665.45,25.82446
2023-02-05 02:40:00,1665.45,1665.45,1663.94,1664.23,37.07355
2023-02-05 02:45:00,1664.23,1664.65,1663.92,1663.92,13.91298
2023-02-05 02:50:00,1663.92,1664.33,1662.71,1662.71,36.71811
2023-02-05 02:55:00,1662.71,1663.09,1661.18,1661.18,51.57567
...,...,...,...,...,...
2023-02-08 13:30:00,1670.10,1672.93,1668.74,1672.17,150.78082
2023-02-08 13:35:00,1672.17,1676.38,1671.66,1676.37,166.36529
2023-02-08 13:40:00,1676.37,1676.37,1674.26,1675.96,84.13427


In [56]:
eth = pd.read_csv ('./eth_actualizado.csv', encoding='unicode_escape')
eth['Time'] = eth['Time'].str.extract('(\d{4}-\d{2}-\d{2}\w\d{2}:\d{2}:\d{2})', expand=True)
eth['Time'] = pd.to_datetime(eth["Time"])
eth.set_index("Time", inplace = True)

In [57]:
eth

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-19 20:55:00,1550.36,1553.69,1549.33,1550.45,2465.53
2023-01-19 21:00:00,1550.45,1557.13,1550.45,1555.90,7096.74
2023-01-19 21:05:00,1555.90,1556.10,1553.50,1554.67,6316.81
2023-01-19 21:10:00,1554.67,1554.91,1552.64,1553.99,2469.26
2023-01-19 21:15:00,1553.99,1556.49,1553.99,1556.49,5341.91
...,...,...,...,...,...
2023-02-08 14:40:00,1676.87,1676.87,1674.20,1676.01,2627.52
2023-02-08 14:45:00,1676.01,1676.27,1673.81,1675.48,815.52
2023-02-08 14:50:00,1675.48,1675.64,1673.43,1674.56,1216.04


In [19]:
#Function to apply some technical indicators from the ta library
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)

In [58]:
apply_technicals(eth)

In [62]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) > 73)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Sell"]= np.where((self.df.trigger) & 
                                    (self.df["K"] + 0.06 < self.df["D"]), 1, 0)

In [63]:
inst = Signals(eth, 0)

inst.decide()

In [64]:
eth[eth.Sell==1]

,Open,High,Low,Close,Volume,K,D,RSI,trigger,Sell
Time,,,,,,,,,,
2023-01-20 17:55:00,1589.63,1590.63,1582.40,1584.91,14203.81,0.641605,0.880535,73.943377,1,1
2023-01-20 18:00:00,1584.91,1589.32,1584.90,1587.08,4483.89,0.692090,0.777898,75.605458,1,1
2023-01-20 18:20:00,1589.07,1591.00,1588.14,1588.14,3522.04,0.480118,0.575723,73.134309,1,1
2023-01-20 21:10:00,1626.86,1627.15,1620.60,1623.38,9964.21,0.810819,0.936940,78.786715,1,1
2023-01-20 21:20:00,1636.60,1636.85,1629.00,1629.00,10189.29,0.637872,0.803588,73.442773,1,1
2023-01-20 22:00:00,1641.09,1653.00,1638.10,1644.00,22531.23,0.335675,0.422394,73.283590,1,1
2023-01-25 21:10:00,1583.66,1584.23,1580.56,1581.70,5775.78,0.853070,0.951023,80.360394,1,1
2023-01-25 23:05:00,1630.60,1641.90,1626.76,1628.43,30483.32,0.873879,0.957960,81.673870,1,1
2023-01-29 02:25:00,1605.92,1613.85,1605.92,1613.68,5221.86,0.676878,0.760674,76.266976,1,1


In [64]:
#The mail addresses and password
sender_address = 'pythontradingbot11@gmail.com'
sender_pass = os.getenv('mail_key')
receiver_address = os.getenv('mail')

message = MIMEMultipart() 
message_SL = MIMEMultipart()
message_TP = MIMEMultipart()
message_RSI = MIMEMultipart()
message_Others = MIMEMultipart()

In [65]:
def strategy_short(qty, open_position = False):
    df= get5minutedata()
    apply_technicals(df)
    inst = Signals(df, 1)
    inst.decide()
    print(f'Current Time is ' + str(df.index[-1]))
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    print(f'Current RSI is ' + str(df.RSI.iloc[-1]))
    print("-----------------------------------------")

    if df.Sell.iloc[-1]:
        mail_content = "ETH Open Short"
        message.attach(MIMEText(mail_content, 'plain'))
        
        # Create SMTP session for sending the mail
        session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
        session_mail.starttls()  # enable security

        # login with mail_id and password
        session_mail.login(sender_address, sender_pass)
        text = message.as_string()
        session_mail.sendmail(sender_address, receiver_address, text)
        session_mail.quit()

        from pybit import usdt_perpetual
        session = usdt_perpetual.HTTP(
        endpoint='https://api.bybit.com',
        api_key = api_key_pw,
        api_secret= api_secret_pw)

        buyprice = round(df.Close.iloc[-1],2)

        print("-----------------------------------------")

        print(f"Buyprice: {buyprice}")

        print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

        order = session.place_active_order(symbol="ETHUSDT",
                                                side="Sell",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=False,
                                                close_on_trigger=False,
                                                take_profit = round(buyprice * 0.93,2),
                                                stop_loss = round(buyprice * 1.03,2))
        print(order)

        eth_order_id = str(order['result']['order_id'])
        print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print(f"Order id: {eth_order_id}") 
        print("---------------------------------------------------")

        open_position = True

    while open_position:
        time.sleep(30)
        from pybit import spot
        session_auth = spot.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)
            
        df = get5minutedata()
        apply_technicals(df)
        print(f"Buyprice: {buyprice}" + '             Close: ' + str(df.Close.iloc[-1]))
        print(f'Target: ' + str(round(buyprice * 0.93, 2)) + "                Stop: " + str(round(buyprice * 1.03, 2)))
        print(f'RSI Target: 53' + '                RSI: ' + str(df.RSI.iloc[-1]))
        print("---------------------------------------------------")

        if df.Close[-1] > buyprice* 1.03:
            print("Closed Position")
            open_position = False

            mail_content_SL = "ETH Short SL"
            message_SL.attach(MIMEText(mail_content_SL, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security

            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_SL.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break

        elif df.Close[-1] < buyprice * 0.93: 
            print("Closed Position")
            open_position = False

            mail_content_TP = "ETH Short TP"
            message_TP.attach(MIMEText(mail_content_TP, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security
            
            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_TP.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break

        elif df.RSI[-1] < 25:
            session = usdt_perpetual.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)

            try: 
                print(session.place_active_order(symbol="ETHUSDT",
                                                side="Buy",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=True,
                                                close_on_trigger=False))  

                print("---------------------------------------------------")
                print("Closed position")
                open_position = False
                mail_content_RSI = "ETH Short Closed - RSI < 25"
                message_RSI.attach(MIMEText(mail_content_RSI, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_RSI.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break

            except: 
                print("Position already closed")
                open_position = False
                
                mail_content_Others = "Position Closed"
                message_Others.attach(MIMEText(mail_content_Others, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_Others.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break



In [66]:
while True: 
    strategy_short(0.7)
    time.sleep(60)

Current Time is 2022-12-20 12:35:00
Current Close is 1213.15
Current RSI is 54.44682199118637
-----------------------------------------
Current Time is 2022-12-20 12:40:00
Current Close is 1215.72
Current RSI is 60.59012230118269
-----------------------------------------
Current Time is 2022-12-20 12:40:00
Current Close is 1215.84
Current RSI is 60.836868276238164
-----------------------------------------


In [ ]:
#ta.volume.VolumeWeightedAveragePrice(high: pandas.core.series.Series, low: pandas.core.series.Series, close: pandas.core.series.Series, volume: pandas.core.series.Series, window: int = 14, fillna: bool = False)